In [8]:
import os
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup

In [9]:
# Filepaths
covid_data = os.path.join("data","cdc_covid.csv")

# Read csv into DataFrame
data = pd.read_csv(covid_data)
data.head()

,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Confirmed Deaths,Probable Deaths,Deaths in Last 7 Days,Death Rate per 100000,Case Rate per 100000 in Last 7 Days,Death Rate per 100K in Last 7 Days
0,Alaska,12118,NaN,NaN,1569,1643,68,NaN,NaN,2,9,30.4,0.0
1,Alabama,180916,155915.0,25001.0,10542,3701,2859,2674.0,185.0,73,58,30.8,0.2
2,Arkansas,104135,NaN,NaN,6596,3455,1782,NaN,NaN,117,59,31.3,0.6
3,American Samoa,0,NaN,NaN,0,0,0,NaN,NaN,0,0,0.0,0.0
4,Arizona,235882,230488.0,5394.0,6396,3289,5865,5578.0,287.0,59,81,12.7,0.1


In [10]:
new_data = data.drop(columns = ['Cases in Last 7 Days', 'Case Rate per 100000', 'Confirmed Deaths', 'Probable Deaths', 'Deaths in Last 7 Days', 'Death Rate per 100000', 'Case Rate per 100000 in Last 7 Days', 'Death Rate per 100K in Last 7 Days'])
new_data.head()

,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Total Deaths
0,Alaska,12118,NaN,NaN,68
1,Alabama,180916,155915.0,25001.0,2859
2,Arkansas,104135,NaN,NaN,1782
3,American Samoa,0,NaN,NaN,0
4,Arizona,235882,230488.0,5394.0,5865


In [22]:
url = "https://covidtracking.com/data/national"
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

In [27]:
results = soup.find_all('td', class_="")
print(len(results))


2493


In [28]:
print(results[0].prettify())

<td class="">
 <span class="_0d5c">
  State (or territory)
 </span>
 <span>
  56
 </span>
</td>

